# RITS Workshop: Data Scraping with Python

In this Jupyter notebook, you will see the code with explanation that will enable you to scrape panting images as wel as the data associated with these paintings.


- Instructor: Noel Konagai
- Email: noel.konagai@nyu.edu

Feel free to contact me for individual follow-up consultations

We start with importing the necessary libraries.

In [1]:
import requests, json, math, urllib.request
import pandas as pd

## Getting the page length

Our page has a "load more" button. Everytime this button is clicked, a new request is sent by our browser to the server, which returns a JSON file. The number of pages can be scraped by sending a request with the first page, and retrieving the 'PageSize' in the response JSON.

In [2]:
base_url = 'https://www.wikiart.org/en/paintings-by-style/'
style = 'pop-art'
query = '?json=2&page='

def page_length():
    r = requests.get(base_url + style + query + '1')
    json_data = json.loads(r.text)
    page_size = json_data['PageSize']
    num_paintings = json_data['AllPaintingsCount']
    num_pages = math.ceil(num_paintings/page_size)
    
    return num_pages, num_paintings, page_size

num_pages, num_paintings, _ = page_length()

## Getting all the necessary data

Write a function that will get the image URL, name of the artist, title, and the year from the given JSON file:

- First, create empty lists for each of these, outside of your function. As we have lots of JSON files to open, we will keep appending new elements to each of these lists.
- the general format to navigate to sub-branches of JSON files is by quoting in paranthesis the key
- example: my_json_data['first_key']
- you can find the appropriate keys if you view the JSON file in your browser
- append each of the data to the apporpriate list

In [3]:
image_urls = []
artist_names = []
titles = []
years = []

def get_info(response_json, page_size):

    for i in range(page_size):

        painting = response_json['Paintings'][i]
        image_urls.append(painting['image'])
        artist_names.append(painting['artistName'])
        titles.append(painting['title'])
        years.append(painting['year'])
        
    return

## Getting all the JSON responses

Write a function that will page from 1 to page_size and get the respective JSON responses:
- write a for function that goes from 0 to num_pages
- your request url should be in the format of base_url plus the number of the page
- send the request with requests.get(your_url)
- create a JSON data with json.loads(your_request.text)
- find the number of paintings in the JSON file
- run the function get_info(your_json_data, num_of_paintings)

In [4]:
from IPython.display import clear_output

def open_all_json():

    for i in range(num_pages):
        clear_output()
        print(str(i + 1) + "/" + str(num_pages) + " pages have been opened")
        request_url = base_url + style + query + str(i + 1)
        r = requests.get(request_url)
        json_data = json.loads(r.text)
        page_size = len(json_data['Paintings'])
        get_info(json_data, page_size)
        
        if i == (num_pages - 1):
            print('Done')
        
    return

In [5]:
open_all_json()

34/34 pages have been opened
Done


Checking whether the number of URLs collected matches the number of paintings.

In [6]:
def sanity_check():

    if len(image_urls) == num_paintings:
        print('The number of URLs match the number of paintings')
    else:
        print('Something went wrong')
        
sanity_check()

The number of URLs match the number of paintings


## Downloading all the image files

Write a function that downloads all the image files:
- loop through the image_urls list
- use urllib.request.urlretrieve(image_url, "path + filename.jpg")
- hint: you can have a variable i = 0, to which you add 1 after every url: you could use this i in str(i).jpg to name your file
- during the workshop, we won't have time to wait for all the files to be downloaded, so we will interrupt the function

In [7]:
# to make sure that non-ascii characters are removed from the url, we pass the url through urllib.parse.quote(your_url)

import urllib.parse as up

def download_images(image_urls, start_imgID, end_imgID):
    
    num = end_imgID - start_imgID
    
    for i in range(num):
        url = image_urls[start_imgID]
        first_part = url[:36]
        second_part = up.quote(url[36:])
        quoted_url = first_part + second_part
        urllib.request.urlretrieve(quoted_url, './' + style + '/' + str(start_imgID) + ".jpg")
        start_imgID += 1
        
    print('Done')
    
    return

def download_images_thread(image_url, imgID):
    
    url = image_url
    first_part = url[:36]
    second_part = up.quote(url[36:])
    quoted_url = first_part + second_part
    urllib.request.urlretrieve(quoted_url, './' + style + '/' + str(imgID) + ".jpg")
    
    return 

In [8]:
import threading
from queue import Queue
import time

print_lock = threading.Lock()
q = Queue()

def threader():
    while True:
        temp = q.get()
        download_images_thread(temp[0], temp[1])
        q.task_done()

for i in range(10):
    t = threading.Thread(target = threader)
    
    t.daemon = True
    t.start()
    
start = time.time()

for i in range(len(image_urls)):
    temp = []
    temp.append(image_urls[i])
    temp.append(i)
    q.put(temp)
    
q.join()

print(time.time() - start)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/noelkonagai/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().co

Exception in thread Thread-10:
Traceback (most recent call last):
  File "/Users/noelkonagai/anaconda/lib/python3.6/urllib/request.py", line 1318, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1239, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1285, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 964, in send
    self.connect()
  File "/Users/noelkonagai/anaconda/lib/python3.6/http/client.py", line 1392, in connect
    super().c

KeyboardInterrupt: 

In [ ]:
download_images(image_urls, 265, len(image_urls))

## Creating a Pandas dataframe with information on the paintings

Pandas is a library typically used for data analytics, but for our puroses it is a really handy tool with which we can easily create dataframes (think of them as spreadsheets). Below is the code to create a CSV file out of all the information that we collected.

In [10]:
def create_dataframe(artist_names, titles, years, image_urls):
    
    # first we create series, think of them as columns in the spreadsheet
    
    s1 = pd.Series(artist_names)
    s2 = pd.Series(titles)
    s3 = pd.Series(years)
    s4 = pd.Series(image_urls)
    
    d = {'artist': s1, 'title': s2, 'year': s3, 'url': s4}
    
    df = pd.DataFrame(data=d)
    
    return df

df = create_dataframe(artist_names, titles, years, image_urls)

# save it into a csv file
df.to_csv(style + '.csv')  